Import modules

In [2]:
from keras.models import Sequential, load_model, Model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization, Input
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
import cv2

%matplotlib inline

Using Theano backend.
DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
   Creating library C:/Users/The Mountain/AppData/Local/Theano/compiledir_Windows-8.1-6.3.9600-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.13-64/tmpyzozh9/265abc51f7c376c224983485238ff1a5.lib and object C:/Users/The Mountain/AppData/Local/Theano/compiledir_Windows-8.1-6.3.9600-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.13-64/tmpyzozh9/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is enabled with initial size: 50.0% of memory, cuDNN 5005)


Load data.

In [3]:
%load_ext autoreload
%autoreload 2
import sys
paths = ['../models', '../dataset', '../util']
for path in paths:
    if path not in sys.path:
        sys.path.append(path)

from bounding_boxes import *
from data_generators import get_data_with_bbox_coords
import dataset
from callbacks import ShowSegmentation

bboxes = get_bounding_boxes('../data')
bboxes = largest_bbox_per_image(bboxes)
train_data = dataset.train
val_data = dataset.validation
 
train_gen = get_data_with_bbox_coords(train_data, '../data/train', bboxes)
val_gen = get_data_with_bbox_coords(val_data, '../data/train', bboxes)
n_labels = len(dataset.labels)

In [4]:
def binning_generator(data_gen, n_bins = 10):
    #
    for images, labels in data_gen:
        batch_size = len(images)
        label_x1 = np.zeros((batch_size, n_bins))
        label_x2 = np.zeros((batch_size, n_bins))
        label_y1 = np.zeros((batch_size, n_bins))
        label_y2 = np.zeros((batch_size, n_bins))
        for i,l, x1, x2, y1, y2 in zip(images, labels, label_x1, label_x2, label_y1, label_y2):
            shape = np.shape(i)
            bin_size = np.ceil(shape[1] / float(n_bins))
            t0 = l[0]
            t1 = l[1]
            l[0] = l[0] // bin_size
            l[1] = l[1] // bin_size
            l[2] = (l[2]+t0) // bin_size
            l[3] = (l[3]+t1) // bin_size
            x1[l[0]] = 1 
            x2[l[1]] = 1 
            y1[l[2]] = 1 
            y2[l[3]] = 1 
        yield images, [label_x1, label_x2, label_y1, label_y2]

Construct model 
Todo: softmax for binning training

In [7]:
def keras_model_conv():
    input1 = Input(shape=(3, 256, 256))
    layer = BatchNormalization(axis=1,  input_shape=(3, 256, 256))(input1)
    layer = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(layer)
    layer = MaxPooling2D(pool_size=(2,2))(layer)
    layer = Dropout(0.50)(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(layer)
    layer = MaxPooling2D(pool_size=(2,2))(layer)
    layer = Dropout(0.50)(layer)
    layer = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(layer)
    layer = MaxPooling2D(pool_size=(2,2))(layer)
    layer = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(layer)
    layer = Dropout(0.50)(layer)
    layer = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(layer)
    layer = MaxPooling2D(pool_size=(2,2))(layer)
    layer = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(layer)
    layer = Dropout(0.50)(layer)
    layer = Flatten()(layer)
    layer = Dense(512, activation='relu')(layer)
    output1 = Dense(10, activation='softmax')(layer)
    output2 = Dense(10, activation='softmax')(layer)
    output3 = Dense(10, activation='softmax')(layer)
    output4 = Dense(10, activation='softmax')(layer)
    model = Model(input=input1, output=[output1, output2, output3, output4])
    model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

    return model

In [8]:
model = keras_model_conv()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 3, 256, 256)   0                                            
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 3, 256, 256)   12          input_2[0][0]                    
____________________________________________________________________________________________________
convolution2d_7 (Convolution2D)  (None, 16, 256, 256)  448         batchnormalization_2[0][0]       
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 16, 128, 128)  0           convolution2d_7[0][0]            
___________________________________________________________________________________________

In [ ]:
model.fit_generator(binning_generator(train_gen), samples_per_epoch=len(train_data),
                    nb_epoch=20, verbose=1,
                    callbacks=None,
                    validation_data=binning_generator(val_gen), nb_val_samples=len(val_data))

Epoch 1/20
3040/3021 [==============================] - 67s - loss: 0.5673 - dense_8_loss: 0.1350 - dense_9_loss: 0.1542 - dense_10_loss: 0.1191 - dense_11_loss: 0.1589 - dense_8_acc: 0.9510 - dense_9_acc: 0.9428 - dense_10_acc: 0.9605 - dense_11_acc: 0.9349 - val_loss: 2.7831 - val_dense_8_loss: 0.6006 - val_dense_9_loss: 0.6905 - val_dense_10_loss: 0.6547 - val_dense_11_loss: 0.8373 - val_dense_8_acc: 0.8099 - val_dense_9_acc: 0.7721 - val_dense_10_acc: 0.8060 - val_dense_11_acc: 0.7305